In [2]:
!pip install transformers

In [3]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 9.0MB/s 


In [4]:
import logging
import os
import argparse
import random
from tqdm import tqdm, trange
import csv

import numpy as np
import sys
import matplotlib.pyplot as plt
import sentencepiece
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler


logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

In [5]:
# Arguments
max_seq_length = 256
train_batch_size = 16
eval_batch_size = 8       
learning_rate = 5e-5      
num_train_epochs = 3            # Epochs - less epochs to be used for BERT
warmup_proportion = 0.1         # How to use?
seed = 42                       # Random seed
local_rank = -1
optimize_on_cpu = False          # Whether to perform optimization and keep the optimizer averages on CPU
fp16 = False                    # Whether to use 16-bit float precision instead of 32-bit
loss_scale = 128                # Loss scaling, positive power of 2 values can improve fp16 convergence
gradient_accumulation_steps = 4 
model_checkpoint = False

In [6]:
#FILES : 

task1_train = 'Task1_train.csv'
task1_dev = 'Task1_dev.csv'
task2_train = 'Task2_train.csv'
task2_dev = 'Task2_dev.csv'

**Necessory FUNCTIONS : **


---




In [7]:
def read_examples(input_file):

    with open(input_file, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        lines = list(reader)

    examples = [
        {
            "article" : line[0],
            "question" : line[1],

            "options" : [line[2], line[3], line[4], line[5], line[6]],
         
            "label" : int(line[7]) 
         } for line in lines[1:]    # we skip the line with the column names
    ]
    return examples



class InputFeatures(object):
    def __init__(self, features , lm_labels, options, label):
        # We didn't stored tokens in features
        self.choices_features = [
            {
                'input_ids': input_ids,
                'input_mask': input_mask,
                'segment_ids': segment_ids
            }
            for _, input_ids, input_mask, segment_ids in features 
        ]

        self.lm_labels = lm_labels
        self.options = options
        self.label = label


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place !
    Both article and question are truncated from end"""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()


In [12]:
def convert_examples_to_features(examples, tokenizer, max_seq_length):
    """Loads a data file into a list of `InputBatch`s."""
    
    features = []
    num_tokens_article = []
    count = 0
    masking_token_id = tokenizer.mask_token_id

    for example_index, example in enumerate(examples):
        # Article Truncated from beginning and not end!
        choices_features = []

        article_tokens = tokenizer.tokenize(example['article'])
        ques_tokens = tokenizer.tokenize(example['question'].replace("@placeholder", tokenizer.mask_token))

        options = []
        for c in example['options']:
            options.append(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(c))[0])
        
        # Truncate Article now        
        _truncate_seq_pair(article_tokens, ques_tokens, max_seq_length - 1)

        # tokens = ques_tokens + [tokenizer.sep_token] + article_tokens + [tokenizer.sep_token]
        # segment_ids = [0] * (len(ques_tokens) + 1) + [1] * (len(article_tokens) + 1)          # Segment Id's 0 -> Ques and 1 -> Article

        tokens = ques_tokens + [tokenizer.sep_token]
        segment_ids = [0] * (len(ques_tokens) + 1) 

        input_ids = tokenizer.convert_tokens_to_ids(tokens)                                   
        input_mask = [1] * len(input_ids)
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += [tokenizer.pad_token_id] * (len(padding))
        input_mask += padding
        segment_ids += padding

        # Heuristic : Loss calculated only for Masked Token!
        lm_labels = [-100 if t_id != masking_token_id else tokenizer.convert_tokens_to_ids(tokenizer.tokenize(example['options'][example['label']]))[0] for t_id in input_ids]

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        assert len(lm_labels) == max_seq_length
        choices_features.append((tokens, input_ids, input_mask, segment_ids))
        # *------------------------------------------------------------------------------------------------------------------*
        # Truncate article from start!
        # article_tokens = tokenizer.tokenize(example['article'])
        # ques_tokens = tokenizer.tokenize(example['question'].replace("@placeholder", "_"))
        # ques_tokens[masked_index] = '[MASK]'

        # choices_features2 = []
        # _truncate_seq_pair2(article_tokens, ques_tokens, max_seq_length - 2)

        # tokens = ques_tokens + ["[SEP]"] + article_tokens + ["[SEP]"]
        # segment_ids = [0] * (len(ques_tokens) + 1) + [1] * (len(article_tokens) + 1)
        # input_ids = tokenizer.convert_tokens_to_ids(tokens)
        # input_mask = [1] * len(input_ids)
        # padding = [0] * (max_seq_length - len(input_ids))
        # input_ids += padding
        # input_mask += padding
        # segment_ids += padding
        # lm_labels = [-1 if t_id != masking_token_id else tokenizer.convert_tokens_to_ids(tokenizer.tokenize(example['options'][example['label']]))[0] for t_id in input_ids]

        # assert len(input_ids) == max_seq_length
        # assert len(input_mask) == max_seq_length
        # assert len(segment_ids) == max_seq_length
        # assert len(lm_labels) == max_seq_length

        # choices_features2.append((tokens, input_ids, input_mask, segment_ids))

        features.append(
            InputFeatures(
                features = choices_features,
                lm_labels = lm_labels,
                options = options,
                label = example['label']
            )
        )
    
    return features



                                                                                                  
                

In [9]:
def copy_optimizer_params_to_model(named_params_model, named_params_optimizer):
    """ Utility function for optimize_on_cpu and 16-bits training.
        Copy the parameters optimized on CPU/RAM back to the model on GPU
    """
    for (name_opti, param_opti), (name_model, param_model) in zip(named_params_optimizer, named_params_model):
        if name_opti != name_model:
            logger.error("name_opti != name_model: {} {}".format(name_opti, name_model))
            raise ValueError
        param_model.data.copy_(param_opti.data)

        
def set_optimizer_params_grad(named_params_optimizer, named_params_model, test_nan=False):
    """ Utility function for optimize_on_cpu and 16-bits training.
        Copy the gradient of the GPU parameters to the CPU/RAMM copy of the model
    """
    is_nan = False
    for (name_opti, param_opti), (name_model, param_model) in zip(named_params_optimizer, named_params_model):
        if name_opti != name_model:
            logger.error("name_opti != name_model: {} {}".format(name_opti, name_model))
            raise ValueError
        if param_model.grad is not None:
            if test_nan and torch.isnan(param_model.grad).sum() > 0:
                is_nan = True
            if param_opti.grad is None:
                param_opti.grad = torch.nn.Parameter(param_opti.data.new().resize_(*param_opti.data.size()))
            param_opti.grad.data.copy_(param_model.grad.data)
        else:
            param_opti.grad = None
    return is_nan



# Define Accuracy check metrics and training utils

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def select_field(features, field):
    return [
            feature.choices_features[0][field]
        for feature in features
    ]
   
    
def classifiction_metric(preds, labels, label_list):
    """ The Metric of classification, input should be numpy format """

    acc = metrics.accuracy_score(labels, preds)

    labels_list = [i for i in range(len(label_list))]

    report = metrics.classification_report(
        labels, preds, labels=labels_list, target_names=label_list, digits=5, output_dict=True)

    return acc, report

Main Code Begins Now : 

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

train_batch_size = int(train_batch_size / gradient_accumulation_steps)
#Initialise seeds
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if n_gpu > 0:
    torch.cuda.manual_seed_all(seed)


Importing Tokenizer and Model From Hugging face library "Transformers"


Following Models have to be tried : 
Reference link for pretrained models : [Hugging Face Pretrained Models](https://huggingface.co/transformers/pretrained_models.html)

1.   XlNet  
2.   Albert  (Expected to work better)  -> v2 version is better than v1
3.   Roberta (Expected to work poorer)
4.   Electra (No idea about it,  Have to study about it!)


Instructions for saving model : 

1. Please download the saved model file into your computer (size might be big though :) )
2. Report the Results in the docs ->. [Doc Link](https://docs.google.com/document/d/1VPKYoi1VahtQgPCs4zMZVYOhy3xHkrq7OLp5fnClmvM/edit?usp=sharing)





For different models import these respectively : 

1.  XLNetTokenizer, XLNetLMHeadModel
2.  AlbertTokenizer,  AlbertForMaskedLM
3.  RobertaTokenizer, RobertaForMaskedLM
4.  ElectraTokenizer, ElectraForMaskedLM



There might be some errors. If so, Please Contact -> Abhishek Mittal, 180022 :)


In [47]:
from transformers import AlbertTokenizer, AlbertForMaskedLM, BertModel, AdamW        
import torch

tokenizer = AlbertTokenizer.from_pretrained('albert-xxlarge-v2')        # Write the tokenizer to be used
model = AlbertForMaskedLM.from_pretrained('albert-xxlarge-v2')    # Write the model to be used

# tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')        # Write the tokenizer to be used
# model = BertForMaskedLM.from_pretrained('bert-large-uncased')    # Write the model to be used

# bert_model = BertModel.from_pretrained('bert-large-uncased')

# tokenizer = ElectraTokenizer.from_pretrained('google/electra-base-discriminator')
# model = ElectraForMaskedLM.from_pretrained('google/electra-base-discriminator')


02/15/2021 10:26:02 - INFO - filelock -   Lock 140211453353096 acquired on /root/.cache/huggingface/transformers/57f3073e5921aab4297e8e1a59ec930b5cc22e13e6ecabad692e1bb4f28d71a2.d6110e25022b713452eb83d5bfa8ae64530995a93d8e694fe52e05aa85dd3a7d.lock


02/15/2021 10:26:03 - INFO - filelock -   Lock 140211453353096 released on /root/.cache/huggingface/transformers/57f3073e5921aab4297e8e1a59ec930b5cc22e13e6ecabad692e1bb4f28d71a2.d6110e25022b713452eb83d5bfa8ae64530995a93d8e694fe52e05aa85dd3a7d.lock


02/15/2021 10:26:03 - INFO - filelock -   Lock 140211453351976 acquired on /root/.cache/huggingface/transformers/d5df78f35f3fedf5dab8dafee426bca04736e16e0a70bd647bd60607a39bf577.f2355728e5c0b990d2832c3ab8751836d17900316f0e874141a85d322e48b3fd.lock


02/15/2021 10:26:03 - INFO - filelock -   Lock 140211453351976 released on /root/.cache/huggingface/transformers/d5df78f35f3fedf5dab8dafee426bca04736e16e0a70bd647bd60607a39bf577.f2355728e5c0b990d2832c3ab8751836d17900316f0e874141a85d322e48b3fd.lock


02/15/2021 10:26:03 - INFO - filelock -   Lock 140211428071688 acquired on /root/.cache/huggingface/transformers/fa287cec8f8154228201180899218fed4f77fa0fd085bfb8a11cd1e1bef85216.4296b379bf943b455a1427c0758ddc23a147936625e7527e40b9087c28d7ae32.lock


02/15/2021 10:26:26 - INFO - filelock -   Lock 140211428071688 released on /root/.cache/huggingface/transformers/fa287cec8f8154228201180899218fed4f77fa0fd085bfb8a11cd1e1bef85216.4296b379bf943b455a1427c0758ddc23a147936625e7527e40b9087c28d7ae32.lock


In [50]:
# Load Saved model (if any)
if model_checkpoint :
    values = torch.load(no_cls_task1)
    model.load_state_dict(values['model'])

# Optimisations
if fp16:
    model.half()
if n_gpu > 1:
    model = torch.nn.DataParallel(model)

# Load Model to device (cuda here)
model.to(device)


AlbertForMaskedLM(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=4096, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((4096,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=4096, out_features=4096, bias=True)
                (key): Linear(in_features=4096, out_features=4096, bias=True)
                (value): Linear(in_features=4096, out_features=4096, bias=

In [14]:
if fp16:
    param_optimizer = [(n, param.clone().detach().to('cpu').float().requires_grad_()) \
                        for n, param in model.named_parameters()]
elif optimize_on_cpu:
    param_optimizer = [(n, param.clone().detach().to('cpu').requires_grad_()) \
                        for n, param in model.named_parameters()]
else:
    param_optimizer = list(model.named_parameters())

no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.0}
    ]


 **Training Code** : 








In [28]:
train_examples = read_examples(task1_train)         # Training Examples
num_train_steps = int(len(train_examples) / train_batch_size / gradient_accumulation_steps * num_train_epochs)
t_total = num_train_steps

# Optimiser is Adam
optimizer = AdamW(optimizer_grouped_parameters,
                         lr = learning_rate)


In [ ]:
print(tokenizer)

PreTrainedTokenizer(name_or_path='bert-large-uncased', vocab_size=30522, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [29]:
global_step = 0
train_features = convert_examples_to_features(
        train_examples, tokenizer, max_seq_length
        )

logger.info("***** Running training *****")
logger.info("  Num examples = %d", len(train_examples))
logger.info("  Batch size = %d", train_batch_size)
logger.info("  Num steps = %d", num_train_steps)

# all_options = torch.tensor([f.options for f in train_features], dtype=torch.long)
all_labels = torch.tensor([f.label for f in train_features], dtype=torch.long)
all_lm_labels = torch.tensor([f.lm_labels for f in train_features], dtype=torch.long)

all_input_ids_1 = torch.tensor(select_field(train_features, 'input_ids'), dtype=torch.long)
all_input_mask_1 = torch.tensor(select_field(train_features, 'input_mask'), dtype=torch.long)
all_segment_ids_1 = torch.tensor(select_field(train_features, 'segment_ids'), dtype=torch.long)

train_data = TensorDataset(all_input_ids_1, all_input_mask_1, all_segment_ids_1, all_labels, all_lm_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=train_batch_size)

model.train()

02/15/2021 09:32:51 - INFO - __main__ -   ***** Running training *****
02/15/2021 09:32:51 - INFO - __main__ -     Num examples = 3227
02/15/2021 09:32:51 - INFO - __main__ -     Batch size = 4
02/15/2021 09:32:51 - INFO - __main__ -     Num steps = 605


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12, elementw

In [ ]:
!nvidia-smi

Sun Jan 17 10:43:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    32W / 250W |   2199MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

TRAIN Loop Snippet : 

In [30]:
correct_ans = 0
wrong_list = []
total_ans = 0

for _ in trange(int(num_train_epochs), desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    
    for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, segment_ids, label, lm_label_ids = batch

        outputs = model(input_ids, attention_mask=input_mask, labels=lm_label_ids, token_type_ids = segment_ids)   #TOkens with labels set to -100 are ignored             
        loss = outputs.loss

        if n_gpu > 1:
            loss = loss.mean() # mean() to average on multi-gpu.
        if fp16 and loss_scale != 1.0:
            # rescale loss for fp16 training
            # see https://docs.nvidia.com/deeplearning/sdk/mixed-precision-training/index.html
            loss = loss * loss_scale
        if gradient_accumulation_steps > 1:
            loss = loss / gradient_accumulation_steps

        loss.backward()

        tr_loss += loss.item()
        nb_tr_steps += 1
        if (step + 1) % gradient_accumulation_steps == 0:
            if fp16 or optimize_on_cpu:
                if fp16 and loss_scale != 1.0:
                    # scale down gradients for fp16 training
                    for param in model.parameters():
                        if param.grad is not None:
                            param.grad.data = param.grad.data / loss_scale
                is_nan = set_optimizer_params_grad(param_optimizer, model.named_parameters(), test_nan=True)
                if is_nan:
                    logger.info("FP16 TRAINING: Nan in gradients, reducing loss scaling")
                    loss_scale = loss_scale / 2
                    model.zero_grad()
                    continue
                optimizer.step()
                copy_optimizer_params_to_model(model.named_parameters(), param_optimizer)
            else:
                optimizer.step()

            train_loss = tr_loss / nb_tr_steps
            print("\tTraining loss : ", train_loss)

            model.zero_grad()
            global_step += 1
            
          
torch.save({
    'model': model.state_dict()
}, 'saved_file_3.txt')
   


Iteration:   0%|          | 4/807 [00:02<08:23,  1.60it/s]

	Training loss :  1.2155051082372665



Iteration:   1%|          | 8/807 [00:04<08:21,  1.59it/s]

	Training loss :  1.2125826105475426



Iteration:   1%|▏         | 12/807 [00:07<08:20,  1.59it/s]

...


Iteration:  99%|█████████▉| 800/807 [09:03<00:04,  1.44it/s]

	Training loss :  1.3074643705785274



Iteration: 100%|█████████▉| 804/807 [09:05<00:02,  1.44it/s]

	Training loss :  1.3054147024652851



Epoch: 100%|██████████| 3/3 [27:15<00:00, 545.28s/it]


In [ ]:
torch.save({
    'model': model.state_dict()
}, 'saved_file_epoch1.txt')


In [51]:
correct_score = 0
wrong_list = []
examples = read_examples('Task_2_dev_full.csv')
model.eval()

AlbertForMaskedLM(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=4096, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((4096,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=4096, out_features=4096, bias=True)
                (key): Linear(in_features=4096, out_features=4096, bias=True)
                (value): Linear(in_features=4096, out_features=4096, bias=

In [ ]:
print(["abhi"] + [tokenizer.mask_token] + [tokenizer.sep_token] + [tokenizer.pad_token])

['abhi', '[MASK]', '[SEP]', '[PAD]']


In [ ]:

inputs = tokenizer("The capital of France is [MASK].", return_tensors="pt")
labels = tokenizer("The capital of France is Paris.", return_tensors="pt")["input_ids"]

inputs = inputs.to('cuda')
labels = labels.to('cuda')

outputs = model(**inputs, labels=labels, output_hidden_states=True)
loss = outputs.loss
logits = outputs.logits

In [ ]:
print(inputs)

{'input_ids': tensor([[ 101, 1996, 3007, 1997, 2605, 2003,  103, 1012,  102]],
       device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


In [ ]:
print(logits[0])

tensor([[ -6.0357,  -5.9746,  -6.0352,  ...,  -5.5175,  -5.4489,  -3.6893],
        [-13.0917, -12.5835, -12.3866,  ...,  -8.3100, -11.5729, -10.4926],
        [ -8.5031,  -9.5911,  -9.4720,  ..., -11.5717,  -9.6129,  -7.2501],
        ...,
        [ -0.6591,  -0.7770,  -0.8077,  ...,  -2.6484,  -2.2071,  -1.9790],
        [ -8.3370,  -7.8262,  -8.5830,  ...,  -5.8269,  -8.2611,  -6.9086],
        [ -7.7844,  -7.5628,  -8.2584,  ...,  -8.0199,  -7.6262,  -8.3779]],
       device='cuda:0', grad_fn=<SelectBackward>)


In [ ]:
print(outputs.hidden_states[-1][0][-3])

tensor([-0.0051,  0.5623, -0.3249,  ..., -0.3349,  0.2096, -0.2982],
       device='cuda:0', grad_fn=<SelectBackward>)


In [ ]:
print(tokenizer.convert_ids_to_tokens(1013))

/


In [ ]:
tokenizer.tokenize("_")

['_']

In [ ]:
list1 = []
list2 = []

In [52]:
with open('outfile.csv', 'w') as f :
    writer = csv.writer(f, delimiter = ',')
    writer.writerow(['Predicted label', 'Option0','Option1','Option2','Option3','Option4'])
    for idx, example in enumerate(examples) :

        article = example['article']
        ques_tokens = tokenizer.tokenize(example['question'].replace("@placeholder", tokenizer.mask_token))        
        tokenized_article = tokenizer.tokenize(article)

        _truncate_seq_pair(tokenized_article, ques_tokens, max_seq_length - 1)

        # tokens =  ques_tokens + [tokenizer.sep_token] + tokenized_article + [tokenizer.sep_token]

        tokens =  ques_tokens + [tokenizer.sep_token]

        masked_index = tokens.index(tokenizer.mask_token)


        candidates = example['options']
        candidates_ids = []
        for c in candidates:
            candidates_ids.append(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(c))[0])


        indexed_tokens = tokenizer.convert_tokens_to_ids(tokens)        
        segments_ids = [0] * (len(ques_tokens) + 1)
        input_mask = [1] * len(indexed_tokens)

        # Zero-pad up to the sequence length.
        padding = [0] * (max_seq_length - len(indexed_tokens))
        indexed_tokens += [tokenizer.pad_token_id] * len(padding)
        input_mask += padding
        segments_ids += padding
        

        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])
        mask_tensors = torch.tensor([input_mask])
        
        mask_tensors = mask_tensors.to(device)
        tokens_tensor = tokens_tensor.to(device)
        segments_tensors = segments_tensors.to(device)


        predictions = model(input_ids = tokens_tensor, attention_mask=mask_tensors, token_type_ids = segments_tensors)
        predictions_candidates = predictions.logits[0, masked_index, candidates_ids]
        answer_idx = torch.argmax(predictions_candidates).item()
        print(answer_idx)
        
        writer.writerow([answer_idx, predictions_candidates[0].item(),predictions_candidates[1].item(),predictions_candidates[2].item(),predictions_candidates[3].item(),predictions_candidates[4].item()])
        print("Correct answer : ", answer_idx, "\tLabel :", example['label'], '\n') 
        if(answer_idx == example['label']):
            correct_score += 1
        else :
            wrong_list.append(idx)



1
Correct answer :  1 	Label : 0 

0
Correct answer :  0 	Label : 0 

3
Correct answer :  3 	Label : 3 

0
Correct answer :  0 	Label : 0 

2
Correct answer :  2 	Label : 2 

2
Correct answer :  2 	Label : 2 

3
Correct answer :  3 	Label : 3 

2
Correct answer :  2 	Label : 2 

3
Correct answer :  3 	Label : 2 

2
Correct answer :  2 	Label : 2 

3
Correct answer :  3 	Label : 3 

3
Correct answer :  3 	Label : 3 

1
Correct answer :  1 	Label : 3 

1
Correct answer :  1 	Label : 1 

2
Correct answer :  2 	Label : 2 

0
Correct answer :  0 	Label : 3 

0
Correct answer :  0 	Label : 2 

3
Correct answer :  3 	Label : 3 

2
Correct answer :  2 	Label : 2 

3
Correct answer :  3 	Label : 3 

2
Correct answer :  2 	Label : 2 

2
Correct answer :  2 	Label : 2 

1
Correct answer :  1 	Label : 1 

3
Correct answer :  3 	Label : 3 

0
Correct answer :  0 	Label : 0 

2
Correct answer :  2 	Label : 2 

4
Correct answer :  4 	Label : 4 

0
Correct answer :  0 	Label : 0 

4
Correct answer :  

In [53]:
accuracy = correct_score / len(examples)  
print("Accuracy :", accuracy)
print("Correct answers :", correct_score) 

print("wrong list items :\n", wrong_list) 


Accuracy : 0.7262044653349001
Correct answers : 618
wrong list items :
 [0, 8, 12, 15, 16, 31, 34, 36, 40, 46, 56, 59, 63, 65, 78, 83, 87, 89, 96, 99, 102, 105, 114, 116, 125, 128, 129, 130, 133, 139, 140, 142, 146, 150, 151, 153, 156, 158, 161, 162, 163, 168, 171, 172, 174, 175, 178, 183, 188, 189, 190, 192, 198, 204, 205, 208, 209, 217, 218, 228, 230, 234, 243, 244, 245, 250, 256, 258, 263, 265, 267, 268, 270, 274, 282, 286, 287, 299, 301, 307, 312, 315, 316, 321, 322, 325, 330, 332, 335, 336, 337, 338, 350, 356, 366, 369, 371, 375, 377, 380, 390, 395, 396, 397, 402, 403, 414, 415, 417, 419, 423, 428, 430, 433, 439, 446, 448, 449, 451, 452, 461, 469, 474, 476, 482, 489, 501, 506, 508, 512, 513, 515, 520, 521, 522, 524, 526, 527, 533, 538, 539, 543, 545, 546, 547, 550, 551, 552, 558, 567, 570, 571, 577, 578, 579, 581, 583, 587, 597, 598, 599, 611, 619, 621, 622, 626, 627, 628, 631, 632, 635, 636, 639, 641, 644, 647, 653, 654, 657, 660, 664, 665, 670, 674, 685, 686, 691, 692, 693, 695,

Material After this is just for practice and is of no use: 


---



---



In [ ]:
for i, item in enumerate(list1):
  if(list2[i] != item):
    print("Yes") 

In [ ]:
tokenizer.encode("Hello, my dog is very <mask> unusual", add_special_tokens=False)

[17, 11368, 19, 94, 2288, 27, 172, 6, 4395]

In [ ]:
# We show how to setup inputs to predict a next token using a bi-directional context.
input_ids = torch.tensor(tokenizer.encode("Hello, my dog is very <mask>", add_special_tokens=False)).unsqueeze(0)  # We will predict the masked token

print(input_ids)

perm_mask = torch.zeros((1, input_ids.shape[1], input_ids.shape[1]), dtype=torch.float)
perm_mask[:, :, -1] = 1.0  # Previous tokens don't see last token i.e <mask> here.

target_mapping = torch.zeros((1, 1, input_ids.shape[1]), dtype=torch.float)  # Shape [1, 1, seq_length] => let's predict one token
target_mapping[0, 0, -1] = 1.0  # Our first (and only) prediction will be the last token of the sequence (the masked token)
input_ids=input_ids.to(device)
perm_mask=perm_mask.to(device)
target_mapping=target_mapping.to(device)
outputs = model(input_ids, perm_mask=perm_mask, target_mapping=target_mapping)
print(np.shape(outputs.logits))
next_token_logits = outputs[0]  # Output has shape [target_mapping.size(0), target_mapping.size(1), config.vocab_size]
print(next_token_logits)
print(torch.argmax(next_token_logits))
# The same way can the XLNetLMHeadModel be used to be trained by standard auto-regressive language modeling.


tensor([[   17, 11368,    19,    94,  2288,    27,   172,     6]])
torch.Size([1, 1, 32000])
tensor([[[-32.9998, -42.5084, -42.8883,  ..., -38.2738, -41.6116, -38.0217]]],
       device='cuda:0', grad_fn=<AddBackward0>)
tensor(172, device='cuda:0')


In [ ]:


input_ids = torch.tensor(tokenizer.encode("Hello, my dog is very <mask>", add_special_tokens=False)).unsqueeze(0)  # We will predict the masked token
labels = torch.tensor(tokenizer.encode("cute", add_special_tokens=False)).unsqueeze(0)

assert labels.shape[0] == 1, 'only one word will be predicted'
perm_mask = torch.zeros((1, input_ids.shape[1], input_ids.shape[1]), dtype=torch.float)
perm_mask[:, :, -1] = 1.0  # Previous tokens don't see last token as is done in standard auto-regressive lm training

target_mapping = torch.zeros((1, 1, input_ids.shape[1]), dtype=torch.float)  # Shape [1, 1, seq_length] => let's predict one token
target_mapping[0, 0, -1] = 1.0  # Our first (and only) prediction will be the last token of the sequence (the masked token)

input_ids=input_ids.to('cuda')
perm_mask=perm_mask.to('cuda')
target_mapping=target_mapping.to('cuda')
labels = labels.to(device)

outputs = model(input_ids, perm_mask=perm_mask, target_mapping=target_mapping, labels=labels)
loss = outputs.loss
next_token_logits = outputs.logits  # Logits have shape [target_mapping.size(0), target_mapping.size(1), config.vocab_size]